In [ ]:
# Climate analysis for Hawaii

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

# Engine connection
engine = create_engine("sqlite:///hawaii.sqlite", echo=False)

# Reflect database into ORM classes
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()


In [ ]:
# Map both tables
Measurements = Base.classes.measurements
Stations = Base.classes.stations

# Create a session
session = Session(engine)

In [ ]:
# Get a list of column names and types
inspector = inspect(engine)

columns = inspector.get_columns('measurements')
#columns
for row in columns:
    print(row["name"], row["type"])


In [ ]:
#Check the table
engine.execute("SELECT * FROM measurements LIMIT 5").fetchall()

In [ ]:
# Check we still have all the expected rows (with dates)
session.query(func.count(Measurements.date)).all()

In [ ]:
# PRECIPITATION ANALYSIS
# Retrieve the last 12 months of precipitation data
import datetime as dt

target_date = dt.date.today() - dt.timedelta(days=365)
#print(target_date)

past_year = session.query(Measurements.date, Measurements.prcp).filter(Measurements.date >= target_date)\
.filter(Measurements.date <= dt.date.today()).all()
#print(past_year)


In [ ]:
# Load results into DataFrame and Plot
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

pastyear_df = pd.DataFrame(past_year, columns=["date", "prcp"])
pastyear_df.set_index("date", inplace=True)
pastyear_df.head()


In [ ]:
# Create a bar chart of Date vs. Precipitation
bar = pastyear_df.plot(kind="bar", figsize=(20,10), legend=False, rot=45, fontsize=12)

# Set the x-axis label
bar.set_xlabel("Date", fontsize=12)

# Set the y-axis label
bar.set_ylabel("Precipitation, inches", fontsize=12)

plt.show()

In [ ]:
# View summary statistics

pastyear_df.describe()

In [ ]:
# STATION ANALYSIS
#Calculate the total number of stations.

#engine.execute("SELECT COUNT(DISTINCT(station)) FROM measurements").fetchall()

# OR - USING A DATAFRAME - nicer output - Number of stations
stations = session.query(Measurements.station, Measurements.tobs).all()
stations_df = pd.DataFrame(stations)
#stations_df.head()

num_stations = stations_df.groupby(["station"]).count()
print("There are " + str(num_stations["tobs"].count()) + " weather stations in Hawaii.")

In [ ]:
#Find the most active stations.
#List the stations and observation counts in descending order
#Which station has the highest number of observations?

#engine.execute("SELECT station, COUNT(station) FROM measurements GROUP BY station ORDER BY station DESC").fetchall()

# OR - USING A DATAFRAME - nicer output - Number of Stations by number of tobs values
stations = session.query(Measurements.station, Measurements.tobs).all()
stations_df = pd.DataFrame(stations)
#stations_df.head()

num_stations = stations_df.groupby(["station"]).count().reset_index().sort_values(["tobs"], ascending=False)
print(num_stations)


In [ ]:
#Retrieve the last 12 months of temperature observation data (tobs).
#Filter by the station with the highest number of observations.

target_date = dt.date.today() - dt.timedelta(days=365)
#print(target_date)

past_year = session.query(Measurements.date, Measurements.station, Measurements.tobs)\
.filter(Measurements.date >= target_date)\
.filter(Measurements.date <= dt.date.today())\
.filter(Measurements.station == "USC00513117").all()

#print(past_year)

#Plot the results as a histogram with bins=12.
temps = [result[2] for result in past_year]
#print(temps)
temps_df = pd.DataFrame(temps)

histo = temps_df.plot.hist(bins=12, figsize=(10,10), legend=False, fontsize=12)

# Set the x-axis label
histo.set_xlabel("Temperature (F)", fontsize=12)

# Set the y-axis label
histo.set_ylabel("Number of Days, Past Year", fontsize=12)

plt.show()


In [ ]:
# TEMPERATURE ANALYSIS
# Write a function called calc_temps that will accept a start date and end date in the format %Y-%m-%d and return the minimum, average, and maximum temperatures for that range of dates.

def calc_temps(start_date, end_date):
    min_temp = session.query(func.min(Measurements.tobs))\
    .filter(Measurements.date >= start_date)\
    .filter(Measurements.date <= end_date).all()

    max_temp = session.query(func.max(Measurements.tobs))\
    .filter(Measurements.date >= start_date)\
    .filter(Measurements.date <= end_date).all()

    avg_temp = session.query(func.avg(Measurements.tobs))\
    .filter(Measurements.date >= start_date)\
    .filter(Measurements.date <= end_date).all()
    
    print("From " + str(start_date) + " to " + str(end_date) + ", the average temperature was " + str(round(avg_temp[0][0],0)) + " degrees, with a minimum of " + str(min_temp[0][0]) + " and a maximum of " + str(max_temp[0][0]) + ".")
    return min_temp, max_temp, avg_temp

In [ ]:
# Use the calc_temps function to calculate the min, avg, and max temperatures for your trip using the matching dates from the previous year (i.e. use "2017-01-01" if your trip start date was "2018-01-01")

start_date = "2016-06-06"
end_date = "2017-06-06"
calc_temps(start_date, end_date)


In [ ]:
# Plot the min, avg, and max temperature from your previous query as a bar chart.
# Use the average temperature as the bar height.
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr).

plt.bar(x=1, height=75, width=1, align="center", color="blue", yerr=8) #, capsize=5)

plt.show()


In [ ]:
# Climate App

from flask import Flask, jsonify

climate_app = Flask(__name__)

@climate_app.route("/")
def home():
    print("Server received request for Home page")
    return "Welcome to the Climate App."

# The homework called this route "/precipitation" but called for temp observations so I made the route to align with the query and data 
@climate_app.route("/api/v1.0/temperature")
def temps():
    import datetime as dt
    target_date = dt.date.today() - dt.timedelta(days=365)
    past_year = session.query(Measurements.date, Measurements.tobs).filter(Measurements.date >= target_date)\
    .filter(Measurements.date <= dt.date.today()).all()
    temp_dict = {Measurements.date:Measurements.tobs}
    all_temps = list(np.ravel(temp_dict))
    return jsonify(all_temps)

if __name__ == "__main__":
    climate_app.run(debug=True)

# NOT DONE YET, FLASK CONNECTION NOT HAPPENING

In [ ]:
@climate_app.route("/api/v1.0/stations")
def home():
    return "Welcome to the Stations Analysis."

@climate_app.route("/api/v1.0/tobs")
def home():
    return "Welcome to the Temperature Analysis."